In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz
import pydotplus

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('./train.csv')
df_test=pd.read_csv('./test.csv')

In [3]:
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [5]:
df.dtypes

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64
registered      int64
count           int64
dtype: object

先自行猜測那些特徵會對租車數量產生影響

1.溫度
2.風速
3.天氣

溫度 小於10度 風速 大於20 天氣 屬於第 3 4 種類 
都會造成租乘數量下降

meaning of weather number
1: Clear, Few clouds, Partly cloudy, Partly cloudy

2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist 

3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds

4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 

In [22]:
df_guess=pd.read_csv('./train.csv')
#溫度
#風速
#天氣
def badFactor(row):
    if row['temp']<=10:
        val=0
    elif row['windspeed']>=20:
        val=0
    elif row['weather']==3 or row['weather']==4:
        val=0
    else:
        val=1
        
    return val

df_guess['guess']=df_guess.apply(badFactor,axis=1)

In [23]:
df_guess.head(20)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,guess
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,0
5,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0,1,1,0
6,2011-01-01 06:00:00,1,0,0,1,9.02,13.635,80,0.0000,2,0,2,0
7,2011-01-01 07:00:00,1,0,0,1,8.20,12.880,86,0.0000,1,2,3,0
8,2011-01-01 08:00:00,1,0,0,1,9.84,14.395,75,0.0000,1,7,8,0
9,2011-01-01 09:00:00,1,0,0,1,13.12,17.425,76,0.0000,8,6,14,1


In [24]:
#利用平均值決定當天數量為高或低
mean=df_guess['count'].mean()
def amount(row):
    if row['count']>=mean:
        val=1
    else:
        val=0
    return val

df_guess['rentAmount']=df_guess.apply(amount,axis=1)

In [25]:
guess_result=df_guess['guess'].values
truth=df_guess['rentAmount'].values

from sklearn.metrics import accuracy_score
accuracy_score(truth,guess_result)

0.5306816094065773

以我自行預想的特徵篩選出來有53percent的準確度，只比隨機猜測好上一些


下面開始運用desicion tree 作分析

In [6]:
#map the weather data to 0-1
weather_map={1:0,2:33,3:66,4:100}
df['weather']=df['weather'].map(weather_map)
df.head(20)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,0,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,0,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,0,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,0,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,0,9.84,14.395,75,0.0000,0,1,1
5,2011-01-01 05:00:00,1,0,0,33,9.84,12.880,75,6.0032,0,1,1
6,2011-01-01 06:00:00,1,0,0,0,9.02,13.635,80,0.0000,2,0,2
7,2011-01-01 07:00:00,1,0,0,0,8.20,12.880,86,0.0000,1,2,3
8,2011-01-01 08:00:00,1,0,0,0,9.84,14.395,75,0.0000,1,7,8
9,2011-01-01 09:00:00,1,0,0,0,13.12,17.425,76,0.0000,8,6,14


In [7]:
#算出當日租借人數 casual與registered的比例
def ratio(row):
    if row['casual']==0:
        val=0
    else:
        val=row['registered']/row['casual']
    return val

df['registered ratio']=df.apply(ratio,axis=1)
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,registered ratio
0,2011-01-01 00:00:00,1,0,0,0,9.84,14.395,81,0.0,3,13,16,4.333333
1,2011-01-01 01:00:00,1,0,0,0,9.02,13.635,80,0.0,8,32,40,4.000000
2,2011-01-01 02:00:00,1,0,0,0,9.02,13.635,80,0.0,5,27,32,5.400000
3,2011-01-01 03:00:00,1,0,0,0,9.84,14.395,75,0.0,3,10,13,3.333333
4,2011-01-01 04:00:00,1,0,0,0,9.84,14.395,75,0.0,0,1,1,0.000000


In [8]:
#將日期分為上班日或非上班日:1為working day 
def work(row):
    if row['holiday']==1:
        val=0
    elif row['holiday']==0 and row['workingday']==0:
        val=0
    else:
        val=1
    return val

df['workingdayOrNot']=df.apply(work,axis=1)

In [9]:
mean=df['count'].mean()

In [10]:
#設定租借數量超過平均為1低於設0

def amount(row):
    if row['count']>=mean:
        val=1
    else:
        val=0
    return val

df['rentAmount']=df.apply(amount,axis=1)

In [11]:
#drop unnecessary feature
df=df.drop(['count','holiday','workingday','datetime',
            'registered','casual'],axis=1)
df.head(20)

,season,weather,temp,atemp,humidity,windspeed,registered ratio,workingdayOrNot,rentAmount
0,1,0,9.84,14.395,81,0.0000,4.333333,0,0
1,1,0,9.02,13.635,80,0.0000,4.000000,0,0
2,1,0,9.02,13.635,80,0.0000,5.400000,0,0
3,1,0,9.84,14.395,75,0.0000,3.333333,0,0
4,1,0,9.84,14.395,75,0.0000,0.000000,0,0
5,1,33,9.84,12.880,75,6.0032,0.000000,0,0
6,1,0,9.02,13.635,80,0.0000,0.000000,0,0
7,1,0,8.20,12.880,86,0.0000,2.000000,0,0
8,1,0,9.84,14.395,75,0.0000,7.000000,0,0
9,1,0,13.12,17.425,76,0.0000,0.750000,0,0


In [12]:
#將data 分為training set 與 valid set
train=df[:8000]
valid=df[8000:]

In [13]:
demand=train['rentAmount'].values
train=train.drop('rentAmount',1)

In [14]:
dtree=DecisionTreeClassifier(max_depth=7)
dtree.fit(train,demand)

dot_data=StringIO()
export_graphviz(dtree,dot_data,filled=True,feature_names=list(train)
                ,class_names=['low demand','high demand']
                ,special_characters=True)
graph=pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_pdf('dtree_bike.pdf')

True

In [15]:
#由此項結果可得知影響租車數量最多的是體感溫度
dtree.feature_importances_

array([0.02477768, 0.00087365, 0.02186678, 0.36961306, 0.19839334,
       0.03264985, 0.29705139, 0.05477424])

In [16]:
y_test=valid['rentAmount'].values
x_test=valid.drop('rentAmount',1)

y_predict=dtree.predict(x_test)

y_predict

array([1, 1, 1, ..., 1, 0, 0], dtype=int64)

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.6884961884961885

經過decision tree 演算法之後，得出的結果準確率可接近70percents，比自行猜測特徵的方法準確不少

我認為兩者的結果差異可能在於沒有用到最相關的特徵，導致準確率只有五成多。

至於如果要改進利用decision tree的效能，我認為如果能將相關度較高的幾個特徵權重提高，應該能得到更好的效果。
